## SETUP

In [2]:
import sys
import os
# add parent directory (where "natsume" lives) to sys.path so I can import natsume
sys.path.append(r"C:\Users\WBS\Desktop\EXOPLANET WORK\006 NATSUME")
import natsume

import ttv_curvefit.curvefit as ttvfit  # custom-written
import ttvfast
from ttvfast import models

from tqdm import tqdm
import pandas as pd
import numpy as np
from scipy.stats import gmean
from scipy.stats import linregress
from astropy import units as u
from astropy.constants import M_earth
import matplotlib.pyplot as plt
from matplotlib.ticker import ScalarFormatter
import matplotlib.colors as mcolors
import seaborn as sns

In [8]:
df = pd.read_csv(r'data_cleaned/merged_2planet.csv')
# Remove all rows with invalid MMR
df = df.dropna(subset=['MMR'])
# Remove all rows with Nbest = 2
bad_stars = df.loc[df['Nbest'] == 2, 'star_name'].unique()
df = df[~df['star_name'].isin(bad_stars)].reset_index(drop=True)

df.head(3)

,name_exoclock,name_exoplanet.eu,star_name,T0_(BJD_TDB),T0_unc.,P_(days),P_unc.,mass,mass_error,orbital_period,...,delta_bic,fap,sampled_f,TTV_strength,planet_pos,period_ratio,MMR,j,N,Delta
0,NaN,HAT-P-26 d,HAT-P-26,NaN,NaN,NaN,NaN,0.020569,0.003319,6.594000,...,NaN,NaN,NaN,NaN,outer,1.557208,3:2,3.0,1.0,0.038139
1,HAT-P-26b,HAT-P-26 b,HAT-P-26,2.457197e+06,0.00007,4.234501,3.200000e-07,0.058500,0.007170,4.234502,...,-2.224506,0.011758,6170.0,no_TTV,inner,1.557208,3:2,3.0,1.0,0.038139
2,HAT-P-27b,HAT-P-27 Ab,HAT-P-27 A,2.457645e+06,0.00014,3.039578,2.400000e-07,0.660000,0.033000,3.039580,...,7.154818,0.186745,8149.0,no_TTV,outer,2.534251,5:2,5.0,3.0,0.013700


## CALC

In [9]:
def analytic_TTV_invert(df_full, star_name):
    # Take only specific star
    df = df_full[df_full.star_name == star_name]

    inner = (df['planet_pos'] == 'inner')
    outer = (df['planet_pos'] == 'outer')

    # Boolean flag
    has_inner_TTV = df.loc[inner, 'Pttv'].notna().any()
    has_outer_TTV = df.loc[outer, 'Pttv'].notna().any()

    # Complex eccentricity
    e_inner = df.loc[inner, 'eccentricity'].iloc[0]
    e_outer = df.loc[outer, 'eccentricity'].iloc[0]
    w_inner = df.loc[inner, 'omega'].iloc[0]
    w_outer = df.loc[outer, 'omega'].iloc[0]
    z = natsume.get_ComplexEccentricities(e_inner, w_inner, e_outer, w_outer)
    # MMR
    MMR = df.loc[outer, 'MMR'].iloc[0]
    # Orbital periods
    Porb_inner = df.loc[inner, 'orbital_period'].iloc[0]
    Porb_outer = df.loc[outer, 'orbital_period'].iloc[0]
    # Stellar mass
    Mstar = df.loc[inner, 'star_mass'].iloc[0]
    
    # Calculate outer mass from inner TTV
    if has_inner_TTV == True:
        Pttv_inner = df.loc[inner, 'Pttv'].iloc[0]
        Attv_inner = df.loc[inner, 'Attv1'].iloc[0]
        TTV_inner = natsume.get_TTVSineCurve(amplitude=Attv_inner, superperiod=Pttv_inner)

        mu_outer = natsume.EstimateOuterMass(
           innerTTV=TTV_inner,
           inner_period=Porb_inner,
           mmr=MMR,
           eccentricity=z,
           outer_period=Porb_outer
        )
        m_outer = (mu_outer * Mstar*u.M_sun).to(u.M_earth).value
        m_outer_err = np.nan
    elif has_inner_TTV == False:
        m_outer, m_outer_err = np.nan, np.nan
        
    if has_outer_TTV == True:
        Pttv_outer = df.loc[outer, 'Pttv'].iloc[0]
        Attv_outer = df.loc[outer, 'Attv1'].iloc[0]
        TTV_outer = natsume.get_TTVSineCurve(amplitude=Attv_outer, superperiod=Pttv_outer)
        
        mu_inner = natsume.EstimateInnerMass(
           innerTTV=TTV_outer,
           outer_period=Porb_outer,
           mmr=MMR,
           eccentricity=z,
           inner_period=Porb_inner
        )
        m_inner = (mu_inner * Mstar*u.M_sun).to(u.M_earth).value
        m_inner_err = np.nan
    elif has_outer_TTV == False:
        m_inner, m_inner_err = np.nan, np.nan

    return m_inner, m_inner_err, m_outer, m_outer_err

In [10]:
df_full = df.copy()
calculated_cols = ['mass_calc', 'mass_calc_err']
df_full[calculated_cols] = np.nan

all_stars = df_full['star_name'].dropna().astype(str).unique()
for star in tqdm(all_stars):
    analytic_TTV_invert(df_full, star)

100%|█████████████████████████████████████████████████████████████████████████████████| 19/19 [00:00<00:00, 433.52it/s]

3:2
5:2
7:3
3:1
3:1
4:1
4:1
5:3
4:1
4:3
5:2
3:1
3:2
3:1
7:3
7:3
4:1
3:1
7:3
